In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedKFold,KFold,train_test_split
from scipy.stats import randint, uniform
from sklearn.metrics import roc_auc_score

import datetime
import random
from operator import itemgetter
import time
import copy

In [3]:
dtrain = xgb.DMatrix('svmlight_new/dtrain.data')
dtest = xgb.DMatrix('svmlight_new/dtest.data')

In [38]:
act_train_data = pd.read_csv("redhat_data_new/act_train_new.csv",dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
act_test_data  = pd.read_csv("redhat_data_new/act_test_new.csv", dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
people_data    = pd.read_csv("redhat_data_new/people.csv", dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])

In [46]:
people_data.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [42]:
act_train_data.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [40]:
act_train_data.shape

(2581352, 15)

In [37]:
dtrain.num_row()

2581352

In [45]:
len(set(act_train_data.people_id.unique())&set(act_test_data.people_id.unique()))

2326

In [22]:
d = dtrain.slice([0,1,2,3,7,8,9])

In [24]:
d.get_label()

array([ 0.,  0.,  0.,  0.,  1.,  1.,  1.], dtype=float32)

In [ ]:
param = {'max_depth':10, 'eta':0.02, 'silent':0, 'objective':'binary:logistic' }
#param['nthread'] = 2
param['eval_metric'] = 'auc'
param['colsample_bytree'] = 0.7
param['subsample'] = 0.7
param['min_child_weight'] = 0
param['booster'] = "gblinear"

In [ ]:
np.random.seed(120)
evals  = [(dtrain,'train')]
num_round = 305
bst = xgb.train(param, dtrain, num_round, evals, verbose_eval=10)

In [ ]:
ypred = bst.predict(dtest)
output = pd.DataFrame({ 'activity_id' : test['activity_id'], 'outcome': ypred })
output.head()
output.to_csv('without_leak.csv', index = False)
!zip subb1.zip without_leak.csv